In [1]:
#Import Libs ( Crypto )
from cryptography.hazmat.primitives.keywrap import aes_key_wrap, aes_key_unwrap
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.padding import PKCS7
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes

In [2]:
#Import Libs
import hashlib
import base64
import os

In [3]:
#File Upload.
path = "/content/Test.txt"

In [4]:
#Files Path Definition
dec_path = path + ".dec"
enc_path = path + ".enc"

In [5]:
#Function to Derive Key
def derive_key(password: str, salt: bytes) -> bytes:

    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    return kdf.derive(password.encode())

In [6]:
#Function to Encrypt the File
def encrypt_file(file_path: str, password: str) -> None:
    with open(file_path, 'rb') as f:
        plaintext = f.read()

    # Key + Salt
    salt = os.urandom(16)
    key = derive_key(password, salt)

    # Start Initialization Vector (IV)
    iv = os.urandom(16)

    # Cipher Setup
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # Padding Apply + AES Apply
    padder = PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(plaintext) + padder.finalize()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()

    # Save a copy of Crypted file
    encrypted_file_path = file_path + '.enc'

    with open(encrypted_file_path, 'wb') as f:
        f.write(salt + iv + ciphertext)

    #Show the Results
    print(f"Encrypted file Path: {encrypted_file_path}")

In [7]:
#Function to Decrypt the File
def decrypt_file(file_path: str, password: str) -> None:
    with open(file_path, 'rb') as f:
        data = f.read()

    # SALT, IV + Cipher File Extraction
    salt = data[:16]
    iv = data[16:32]
    ciphertext = data[32:]

    # Key Derivation
    key = derive_key(password, salt)

    # Decrypt Setup
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()

    # Padding Removing + Decrypt Apply
    padded_data = decryptor.update(ciphertext) + decryptor.finalize()
    unpadder = PKCS7(algorithms.AES.block_size).unpadder()
    plaintext = unpadder.update(padded_data) + unpadder.finalize()

    # Save a copy of Decrypted file
    decrypted_file_path = file_path.replace('.enc', '.dec')
    with open(decrypted_file_path, 'wb') as f:
        f.write(plaintext)

    #Show the Results
    print(f"Decrypted file Path: {decrypted_file_path}")

In [8]:
#Application Example ( Path, Password )
encrypt_file(path, '1234')

Encrypted file Path: /content/Test.txt.enc


In [9]:
#Decrypt Application ( Path, Password )
decrypt_file(enc_path, '1234')

Decrypted file Path: /content/Test.txt.dec


In [10]:
#Get the Values - Original File
with open(path, 'r', encoding='utf-8') as file:
            org = file.read()

In [11]:
#Get the Values - Encrypted File (Encoding with latin-1)
with open(enc_path, 'r', encoding='latin-1') as file:
            enc = file.read()

In [12]:
#Get the Values - Decrypted File (Encoding with utf-8)
with open(dec_path, 'r', encoding='utf-8') as file:
            dec = file.read()

In [13]:
#Show the Results
print("Original Content: " + org)
print("Encrypted Content: " + enc)
print("Decrypted Content: " + dec)

Original Content: Test Content

Encrypted Content: ,«í4ÚM­4T&fÁþw¼B©«N¡v,,¹p~tJºÛ9þê
Decrypted Content: Test Content

